In [35]:
#!conda list

In [36]:
import pygrib
import pandas as pd
import numpy as np

In [37]:
gribName="20210610_0616_9e528005"
cyc_name=str.upper("Carlos")
grbs = pygrib.open('.\\Data\\'+gribName+'.grib')

In [38]:
grb=grbs[1:]
grb.append(grbs[len(grb)+1])

In [39]:
#selected_grbs=grbs.select(shortName="u",typeOfLevel='isobaricInhPa',level=[500,700,800,925,1000])
#selected_grbs[0]

In [41]:
numChannels=12
gridShape=(221,381)
framesPerDay=24
isoLvls=[500,700,850,925,1000]
numIsoLvls=len(isoLvls)
numFrames=len(grb)
numDays=numFrames//(numChannels*framesPerDay*numIsoLvls)
#INDEX=Frame*5*12+IsoLvl*12+Channel (tot=12*5*24*numDays)
assert numFrames%(numChannels*framesPerDay*numIsoLvls)==0 #Having integer number of days
assert grb[1].values.shape==gridShape #Check grid shape matches

In [7]:
firstFrame=grb[0]['day']

In [8]:
#grb[0].keys()

In [42]:
#grb[0:13]

In [10]:
arr=np.zeros(shape=(numDays*framesPerDay,numIsoLvls,numChannels,*gridShape),dtype="float32")
print(arr.shape)

def parseIndices(i):
    timeLoc=i//(numChannels*numIsoLvls)
    isoLoc=(i-timeLoc*(numChannels*numIsoLvls))//numChannels
    channelLoc=i-timeLoc*(numChannels*numIsoLvls)-isoLoc*numChannels
    return timeLoc,isoLoc,channelLoc

#INDEX=Frame*60+IsoLvl*12+Channel (tot=12*5*24*numDays)
for i,v in enumerate(grb):
    arr[*parseIndices(i)] = v.values.astype("float32")


(168, 5, 12, 221, 381)


In [11]:
parseIndices(247)

(4, 0, 7)

In [12]:
grb[247].values

array([[255.15921021, 255.20413208, 255.25784302, ..., 261.86038208,
        261.82131958, 261.75881958],
       [255.78030396, 255.83792114, 255.90237427, ..., 261.72073364,
        261.69924927, 261.65628052],
       [256.40432739, 256.46585083, 256.53225708, ..., 261.65139771,
        261.64456177, 261.63870239],
       ...,
       [269.43557739, 269.40921021, 269.38870239, ..., 270.67288208,
        270.59866333, 270.53421021],
       [269.62112427, 269.57913208, 269.54397583, ..., 270.75881958,
        270.71487427, 270.66116333],
       [269.80862427, 269.76565552, 269.72659302, ..., 270.74514771,
        270.74612427, 270.72366333]])

In [13]:
del grb
del grbs

In [14]:
arr[0,0,1]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [15]:
import numpy as np

# Assuming `arr` is your ndarray with the shape (numDays*framesPerDay, numIsoLvls, numChannels, *gridShape)
def normalize_channels(data):
    # data.shape should be (numDays*framesPerDay, numIsoLvls, numChannels, height, width)

    # Initialize a new array to hold the normalized data
    normalized_data = np.zeros_like(data, dtype=np.float32)
    
    # Iterate over each channel to normalize
    for channel in range(data.shape[2]):  # Assuming the third index is for channels
        # Extract the channel data
        channel_data = data[:, :, channel, :, :]
        
        # Compute the min and max
        min_val = np.min(channel_data)
        max_val = np.max(channel_data)
        
        # Avoid division by zero in case all values in the channel are the same
        if max_val > min_val:
            # Normalize the channel
            normalized_data[:, :, channel, :, :] = (channel_data - min_val) / (max_val - min_val)
        else:
            # If max equals min, set the normalized data to zero or handle appropriately
            normalized_data[:, :, channel, :, :] = 0  # Or set it to a neutral value like 0.5 if more appropriate

    return normalized_data

# Apply the normalization to your data
normalized_arr = normalize_channels(arr)

In [16]:
import numpy as np

# Assuming normalized_arr is your ndarray
# Example shape of normalized_arr is (numDays*framesPerDay, numIsoLvls, numChannels, height, width)
# We need to transform it to (numDays*framesPerDay, height, width, numChannels*numIsoLvls)

# First, let's transpose the axes to bring all spatial dimensions next to each other
# Transpose (0, 3, 4, 1, 2) means:
# 0 -> 0 (keep numDays*framesPerDay at the first axis)
# 1 -> 3 (move the first spatial dimension to second place)
# 2 -> 4 (move the second spatial dimension to third place)
# 3 -> 1 (move numIsoLvls to the last but one)
# 4 -> 2 (move numChannels to the last position)
transposed_arr = np.transpose(normalized_arr, (0, 3, 4, 1, 2))

# Now, we need to merge the last two dimensions (numIsoLvls and numChannels)
# Calculate the new dimension
new_last_dim = transposed_arr.shape[3] * transposed_arr.shape[4]

# Reshape to merge the last two dimensions
inTensor = transposed_arr.reshape(transposed_arr.shape[0], transposed_arr.shape[1], transposed_arr.shape[2], new_last_dim)

# reshaped_arr now has the shape (numDays*framesPerDay, height, width, numChannels*numIsoLvls)

In [17]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Concatenate
from tensorflow.keras.models import Model

# Assuming each "frame" is an independent input not wrapped in TimeDistributed
input_layer = Input(shape=(13, *gridShape, 60))  # Adjust based on your exact gridShape and channel info

# CNN layers applied independently to each frame (you might need a loop or custom layer here)
# Simplified version: Applying CNN to the first frame for illustration (adjust this part!)
cnn_out = Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(input_layer[:,0,:,:,:])
cnn_out = MaxPooling2D(pool_size=(3, 3))(cnn_out)
cnn_out = Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(cnn_out)
cnn_out = MaxPooling2D(pool_size=(3, 3))(cnn_out)
cnn_out = Conv2D(filters=128, kernel_size=(3, 3), activation='relu')(cnn_out)
cnn_out = MaxPooling2D(pool_size=(2, 2))(cnn_out)
cnn_out = Flatten()(cnn_out)

# Scalar inputs for the best track data at present (shape=(6,))
best_track_input = Input(shape=(6,))

# Concatenate the scalar values with the output of the CNN
combined_inputs = Concatenate(axis=-1)([cnn_out, best_track_input])

# Dense layers for final prediction
dense_out = Dense(units=100, activation='relu')(combined_inputs)
output = Dense(units=4)(dense_out)  # Predicting values for the cyclone's next location

# Compile the model
model = Model(inputs=[input_layer, best_track_input], outputs=output)
model.compile(optimizer='adam', loss='mean_squared_error')

model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 13, 221,   │          0 │ -                 │
│ (InputLayer)        │ 381, 60)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item (GetItem)  │ (None, 221, 381,  │          0 │ input_layer[0][0] │
│                     │ 60)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 219, 379,  │     17,312 │ get_item[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 73, 126,   │          0 │ conv2d[0][0]      │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 71, 124,   │     18,496 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 23, 41,    │          0 │ conv2d_1[0][0]    │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 21, 39,    │     73,856 │ max_pooling2d_1[… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_2     │ (None, 10, 19,    │          0 │ conv2d_2[0][0]    │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 24320)     │          0 │ max_pooling2d_2[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 6)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 24326)     │          0 │ flatten[0][0],    │
│ (Concatenate)       │                   │            │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 100)       │  2,432,700 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 4)         │        404 │ dense[0][0]       │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 2,542,768 (9.70 MB)

 Trainable params: 2,542,768 (9.70 MB)

 Non-trainable params: 0 (0.00 B)

In [18]:
best_Track=pd.read_csv("BestTrack_Trimmed.csv")

In [19]:
firstFrame

10

In [20]:
best_Track['ISO_TIME']=pd.to_datetime(best_Track['ISO_TIME'])

In [21]:
starttime=pd.to_datetime(gribName[:8],format='%Y%m%d')
endtime=starttime+pd.Timedelta(days=numDays)

In [22]:
fBesttrack=best_Track[(best_Track['ISO_TIME']>=starttime)&(best_Track['ISO_TIME']<=endtime)&(best_Track['LAT']>=0.0)&(best_Track['LAT']<=90.0)&(best_Track['LON']>=-170.0)&(best_Track['LON']<=-80.0)]

In [23]:
fBesttrack=fBesttrack[fBesttrack['NAME']==cyc_name]

In [24]:
selected_columns = fBesttrack[['USA_WIND', 'USA_PRES', 'LAT', 'LON','ISO_TIME']]
selected_columns=selected_columns[selected_columns['ISO_TIME'].dt.hour%3==0]
def normalize(value, min_val, max_val):
    return (value - min_val) / (max_val - min_val)

# Define the ranges
lat_min, lat_max = 0, 40
lon_min, lon_max = -170, -80
wind_min, wind_max = 20, 200
pressure_min, pressure_max = 860, 1020

# Normalize the data
selected_columns['LAT'] = normalize(selected_columns['LAT'], lat_min, lat_max)
selected_columns['LON'] = normalize(selected_columns['LON'], lon_min, lon_max)
selected_columns['USA_WIND'] = normalize(selected_columns['USA_WIND'].astype(float), wind_min, wind_max)
selected_columns['USA_PRES'] = normalize(selected_columns['USA_PRES'].astype(float), pressure_min, pressure_max)
selected_columns['ISO_DATE'] = selected_columns['ISO_TIME'].dt.dayofyear/365
selected_columns['ISO_TIME'] = selected_columns['ISO_TIME'].dt.hour/24

In [25]:
import datetime
starttimeb=fBesttrack['ISO_TIME'].iloc[0]
endtimeb=fBesttrack['ISO_TIME'].iloc[-1]
assert (endtimeb-starttimeb)/datetime.timedelta(seconds=3*3600) == len(selected_columns)-1 #same frame count

besttrack_ndarr=selected_columns.to_numpy().astype("float32")

In [26]:
# import pandas as pd
# df = pd.read_csv('ibtracs.EP.list.v04r00.csv')

In [27]:
# df_trim=df[['SID','NAME','SEASON','NUMBER','NATURE','ISO_TIME','LAT','LON','WMO_WIND','WMO_PRES','USA_RECORD','USA_STATUS','USA_WIND','USA_PRES','USA_GUST']][1:]
# df_trim['SEASON']=pd.to_numeric(df_trim['SEASON'])
# df_trim=df_trim[df_trim['SEASON']>=2015]

In [28]:
#df_trim.to_csv('BestTrack_Trimmed.csv')

In [29]:
#import cdsapi
#c = cdsapi.Client()
#c.retrieve()

In [30]:
(inTensor.size*inTensor.itemsize)/(1024*1024*1024)#in GB

3.1618255376815796

In [31]:
startFrameinA=int((starttimeb-starttime)/datetime.timedelta(hours=1))
endFrameinA=int((endtimeb-starttime)/datetime.timedelta(hours=1))

In [32]:
import numpy as np
import tensorflow as tf

def Bdx_to_Adx(bdx, startFrameinA):
    """ Converts an index in best track data to its corresponding index in inTensor. """
    return bdx * 3 + startFrameinA

def data_generator(inTensor, besttrack_ndarr, startFrameinA, endFrameinA, scope=12, forecast_horizon=6):
    """
    Generator function to yield individual training data samples and labels.
    """
    # Calculate the number of samples available for generation
    num_samples = len(besttrack_ndarr) - forecast_horizon // 3
    indices = np.arange(0, num_samples)
    np.random.shuffle(indices)  # Shuffle indices for random sampling
    
    while True:  # Loop indefinitely for continuous generation
        for idx in indices:
            # Calculate starting frame index for inTensor based on best track data index
            start_idx = Bdx_to_Adx(idx, startFrameinA) - scope
            end_idx = Bdx_to_Adx(idx, startFrameinA) + 1
            
            # Generate input frames from inTensor
            cnn_input = np.stack(inTensor[start_idx:end_idx])
            
            # Get corresponding best track input
            track_input = besttrack_ndarr[idx]
            
            # Prepare target output based on forecast horizon
            target = besttrack_ndarr[idx + forecast_horizon // 3][:4]
            
            if cnn_input is None or track_input is None or target is None:
                print("None found in generator output!")
                continue  # or handle it differently as needed
            
            # print("Yielding data:", cnn_input.shape, track_input.shape, target.shape)
            # Yield the individual data sample and target
            yield ((cnn_input, track_input), target)

# Define output signatures to specify the shapes and types of outputs expected
output_signature = (
    (
        tf.TensorSpec(shape=(13, *gridShape, 60), dtype=tf.float32),  # CNN input shape
        tf.TensorSpec(shape=(6,), dtype=tf.float32)  # Best track input shape
    ),
    tf.TensorSpec(shape=(4,), dtype=tf.float32)  # Output target shape
)

# Create a TensorFlow dataset from the generator, specifying batch size within the dataset pipeline
dataset = tf.data.Dataset.from_generator(
    generator=lambda: data_generator(inTensor, besttrack_ndarr, startFrameinA, endFrameinA),
    output_signature=output_signature
).batch(3)  # Define batch size here

# Example usage: Ready for model training
# model.fit(dataset, epochs=10)

In [33]:
try:
    model.fit(x=dataset, steps_per_epoch=10, epochs=4)
except Exception as e:
    import traceback
    traceback.print_exc()

Epoch 1/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 11s 853ms/step - loss: 0.2025
Epoch 2/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 8s 793ms/step - loss: 0.0042
Epoch 3/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 8s 792ms/step - loss: 0.0017
Epoch 4/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 8s 755ms/step - loss: 0.0018
Epoch 5/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 8s 780ms/step - loss: 0.0011
Epoch 6/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 8s 758ms/step - loss: 0.0012
Epoch 7/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 8s 771ms/step - loss: 7.3896e-04
Epoch 8/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 8s 753ms/step - loss: 6.6126e-04
Epoch 9/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 8s 777ms/step - loss: 5.1188e-04
Epoch 10/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 8s 785ms/step - loss: 4.4176e-04


In [ ]:
def load_grib(fname,cname):
    global numFrames,numDays,cyc_name
    gribName=fname
    cyc_name=str.upper(cname)
    grbs = pygrib.open('.\\Data\\'+gribName+'.grib')
    grb=grbs[1:]
    grb.append(grbs[len(grb)+1])
    
    numFrames=len(grb)
    numDays=numFrames//(numChannels*framesPerDay*numIsoLvls)
    #INDEX=Frame*5*12+IsoLvl*12+Channel (tot=12*5*24*numDays)
    assert numFrames%(numChannels*framesPerDay*numIsoLvls)==0 #Having integer number of days
    assert grb[1].values.shape==gridShape #Check grid shape matches

        
    for i,v in enumerate(grb):
        arr[*parseIndices(i)] = v.values.astype("float32")

    del grb
    del grbs
    
    
    normalized_arr = normalize_channels(arr)
    transposed_arr = np.transpose(normalized_arr, (0, 3, 4, 1, 2))

    # Now, we need to merge the last two dimensions (numIsoLvls and numChannels)
    # Calculate the new dimension
    new_last_dim = transposed_arr.shape[3] * transposed_arr.shape[4]

    # Reshape to merge the last two dimensions
    inTensor = transposed_arr.reshape(transposed_arr.shape[0], transposed_arr.shape[1], transposed_arr.shape[2], new_last_dim)

    best_Track=pd.read_csv("BestTrack_Trimmed.csv")
    starttime=pd.to_datetime(gribName[:8],format='%Y%m%d')
    endtime=starttime+pd.Timedelta(days=numDays)
    fBesttrack=best_Track[(best_Track['ISO_TIME']>=starttime)&(best_Track['ISO_TIME']<=endtime)&(best_Track['LAT']>=0.0)&(best_Track['LAT']<=90.0)&(best_Track['LON']>=-170.0)&(best_Track['LON']<=-80.0)&(best_Track['NAME']==cyc_name)]
    selected_columns = fBesttrack[['USA_WIND', 'USA_PRES', 'LAT', 'LON','ISO_TIME']]
    selected_columns=selected_columns[selected_columns['ISO_TIME'].dt.hour%3==0]

    # Normalize the besttrack data
    selected_columns['LAT'] = normalize(selected_columns['LAT'], lat_min, lat_max)
    selected_columns['LON'] = normalize(selected_columns['LON'], lon_min, lon_max)
    selected_columns['USA_WIND'] = normalize(selected_columns['USA_WIND'].astype(float), wind_min, wind_max)
    selected_columns['USA_PRES'] = normalize(selected_columns['USA_PRES'].astype(float), pressure_min, pressure_max)
    selected_columns['ISO_DATE'] = selected_columns['ISO_TIME'].dt.dayofyear/365
    selected_columns['ISO_TIME'] = selected_columns['ISO_TIME'].dt.hour/24

    starttimeb=fBesttrack['ISO_TIME'].iloc[0]
    endtimeb=fBesttrack['ISO_TIME'].iloc[-1]
    assert (endtimeb-starttimeb)/datetime.timedelta(seconds=3*3600) == len(selected_columns)-1 #same frame count
    besttrack_ndarr=selected_columns.to_numpy().astype("float32")

    startFrameinA=int((starttimeb-starttime)/datetime.timedelta(hours=1))
    endFrameinA=int((endtimeb-starttime)/datetime.timedelta(hours=1))

    dataset = tf.data.Dataset.from_generator(
        generator=lambda: data_generator(inTensor, besttrack_ndarr, startFrameinA, endFrameinA),
        output_signature=output_signature
    ).batch(3)  # Define batch size here

In [ ]:
model.fit(x=dataset, steps_per_epoch=10, epochs=4)